In [1]:
#!pip install timm
#!pip install ipywidgets
#!pip install optuna plotly
#!pip install kaleido
#!pip install matplotlib

# new model - coatnet
hybrid architecture - Convolution + Attention.

In [3]:
import torch
import numpy as np
import pandas as pd
import evaluate
import timm
import shutil
import os
import safetensors.torch
from torch.utils.data import DataLoader
from datasets import load_from_disk
from transformers import TrainingArguments, Trainer, set_seed
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
from torchvision.transforms import (
    Compose, Resize, CenterCrop, ToTensor, Normalize, 
    RandomHorizontalFlip, RandomResizedCrop, TrivialAugmentWide, InterpolationMode
)

In [ ]:
SEED = 100
MODEL_NAME = "coatnet_0_rw_224"
DATA_PATH = "processed_bird_data"
OUTPUT_DIR = "coatnet_aug_experiment"
NUM_EPOCHS = 30
BATCH_SIZE = 32

# Hyperparameters (Optimized)
LEARNING_RATE = 5.9e-4
WEIGHT_DECAY = 0.065

# Config for augmentation
mixup_args = {
    # Strength of blending
    'mixup_alpha': 0.8,
    # Strength of cutting/pasting
    'cutmix_alpha': 1.0,
    # Apply it to 100% of batches
    'prob': 1.0,
    # 50% chance of Mixup vs 50% CutMix
    'switch_prob': 0.5,
    # Apply to whole batch
    'mode': 'batch',
    # Smoothing
    'label_smoothing': 0.1,
    'num_classes': 200
}

mixup_fn = Mixup(**mixup_args)

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

set_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available(): device = "mps"
print(f"Using device: {device} | Seed: {SEED}")

Using device: mps | Seed: 100


Applying Aggressive Data Augmentation to prevent overfitting:

1. Does random resize/zoom (scale 0.8–1.0). Forces the model to recognize a bird by looking at its specific feature (e.g. the head, wing, etc.) instead of the background (e.g. tress).
2. Incorporates horizontal flipping to double the training data.

In [4]:
dataset = load_from_disk(DATA_PATH)

normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

_train_transforms = Compose([
    RandomResizedCrop(224, scale=(0.8, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
    TrivialAugmentWide(),
    ToTensor(),
    normalize,
])

_val_transforms = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    normalize,
])

def train_transforms(batch):
    batch["pixel_values"] = [_train_transforms(image.convert("RGB")) for image in batch["image"]]
    if "image" in batch:
        del batch["image"]
    return batch

def val_transforms(batch):
    batch["pixel_values"] = [_val_transforms(image.convert("RGB")) for image in batch["image"]]
    if "image" in batch:
        del batch["image"]
    return batch

dataset["train"].set_transform(train_transforms)
dataset["validation"].set_transform(val_transforms)

# Custom trainer 
class SteroidTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epoch_train_loss = 0.0
        self.epoch_train_acc = 0.0
        self.epoch_steps = 0

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        pixel_values = inputs.get("pixel_values")
        labels = inputs.get("labels")
        
        mixup_is_active = model.training and mixup_fn is not None
        
        original_labels = labels.clone().detach()
        
        # Mixup/CutMix Transformation (labels become soft targets)
        if mixup_is_active:
            pixel_values, labels = mixup_fn(pixel_values, labels)
        
        logits = model(pixel_values)
        
        loss_fct = SoftTargetCrossEntropy() if mixup_is_active else torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        
        if model.training:
            with torch.no_grad():
                preds = torch.argmax(logits, dim=-1)
                
                current_acc = (preds == original_labels).float().mean().item()
                
                self.epoch_train_loss += loss.item()
                self.epoch_train_acc += current_acc
                self.epoch_steps += 1
                
                if self.epoch_steps % 20 == 0:
                    current_epoch_float = self.state.epoch if self.state.epoch is not None else 0
                    print(f" >> Epoch: {current_epoch_float:.2f} | Batch: {self.epoch_steps} | Curr Loss: {loss.item():.4f}")
        
        return (loss, logits) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        inputs = self._prepare_inputs(inputs)
        labels = inputs.pop("labels", None)
        
        with torch.no_grad():
            outputs = model(inputs["pixel_values"])
            loss = None
            if not prediction_loss_only and labels is not None:
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(outputs, labels)

        return (loss, outputs, labels)

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        
        avg_train_loss = self.epoch_train_loss / self.epoch_steps if self.epoch_steps > 0 else 0
        avg_train_acc = self.epoch_train_acc / self.epoch_steps if self.epoch_steps > 0 else 0
        
        val_loss = metrics.get(f"{metric_key_prefix}_loss", 0.0)
        val_acc = metrics.get(f"{metric_key_prefix}_accuracy", 0.0)
        
        print("\n" + "="*80)
        print(f" EPOCH {self.state.epoch:.2f} RESULTS:")
        print(f" Training Loss:   {avg_train_loss:.4f} | Training Acc:   {avg_train_acc*100:.2f}% (Hard Labels)")
        print(f" Validation Loss: {val_loss:.4f}       | Validation Acc: {val_acc*100:.2f}% (Actual performance)")
        print("="*80 + "\n")
        
        # Reset counters for the next epoch
        self.epoch_train_loss = 0.0
        self.epoch_train_acc = 0.0
        self.epoch_steps = 0
        return metrics

# Training
Initializing CoAtNet model with random weights, no pretraining. 

It uses standard Convolutional layers in the early stages to extract low-level features (edges, textures), and then uses Transformer layers in the final stages to understand the global shape of the bird. Using Bayesian optimisation.

In [ ]:
print(f"Initializing {MODEL_NAME} from scratch.")
model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=200)
model.to(device)

accuracy = evaluate.load("accuracy")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE, 
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=0.1,
    
    dataloader_drop_last=True,
    remove_unused_columns=False,
    eval_strategy="epoch",
    
    logging_strategy="steps", 
    logging_steps=50,
    
    save_strategy="epoch",
    load_best_model_at_end=True,     
    metric_for_best_model="accuracy",
    save_total_limit=1,
    disable_tqdm=False,
    report_to="none",
    seed=SEED
)

trainer = SteroidTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

print("Starting Training (Mixup + CutMix + TrivialAugment).")
trainer.train()

final_path = f"{OUTPUT_DIR}/best_aug_model"
trainer.save_model(final_path)

Initializing coatnet_0_rw_224 from scratch.
Starting Training (Mixup + CutMix + TrivialAugment).


/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,5.327500,5.228301,0.010417
2,5.271900,5.146622,0.019097
3,5.221500,5.099582,0.027778
4,5.149300,5.016776,0.015625
5,5.123400,4.921926,0.031250
6,5.085200,4.838113,0.022569
7,5.060800,4.768239,0.038194
8,4.990300,4.743011,0.041667
9,4.921800,4.586624,0.043403
10,4.943800,4.590762,0.055556


 >> Epoch: 0.18 | Batch: 20 | Curr Loss: 5.3284
 >> Epoch: 0.38 | Batch: 40 | Curr Loss: 5.1422
 >> Epoch: 0.57 | Batch: 60 | Curr Loss: 5.2223
 >> Epoch: 0.76 | Batch: 80 | Curr Loss: 5.2385
 >> Epoch: 0.95 | Batch: 100 | Curr Loss: 5.4777

 EPOCH 1.00 RESULTS:
 Training Loss:   5.3406 | Training Acc:   1.14% (Hard Labels)
 Validation Loss: 5.2283       | Validation Acc: 1.04% (Actual performance)



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

 >> Epoch: 27.18 | Batch: 20 | Curr Loss: 4.8756
 >> Epoch: 27.38 | Batch: 40 | Curr Loss: 4.6545
 >> Epoch: 27.57 | Batch: 60 | Curr Loss: 3.7881
 >> Epoch: 27.76 | Batch: 80 | Curr Loss: 4.3791
 >> Epoch: 27.95 | Batch: 100 | Curr Loss: 4.6723

 EPOCH 28.00 RESULTS:
 Training Loss:   4.5690 | Training Acc:   9.25% (Hard Labels)
 Validation Loss: 3.9250       | Validation Acc: 13.37% (Actual performance)



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 28.18 | Batch: 20 | Curr Loss: 4.4772
 >> Epoch: 28.38 | Batch: 40 | Curr Loss: 4.1977
 >> Epoch: 28.57 | Batch: 60 | Curr Loss: 4.8222
 >> Epoch: 28.76 | Batch: 80 | Curr Loss: 4.7327
 >> Epoch: 28.95 | Batch: 100 | Curr Loss: 4.9990

 EPOCH 29.00 RESULTS:
 Training Loss:   4.5129 | Training Acc:   8.05% (Hard Labels)
 Validation Loss: 3.8950       | Validation Acc: 13.89% (Actual performance)



/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


 >> Epoch: 29.18 | Batch: 20 | Curr Loss: 4.1156
 >> Epoch: 29.38 | Batch: 40 | Curr Loss: 3.9390
 >> Epoch: 29.57 | Batch: 60 | Curr Loss: 3.9203
 >> Epoch: 29.76 | Batch: 80 | Curr Loss: 4.7210
 >> Epoch: 29.95 | Batch: 100 | Curr Loss: 4.1404

 EPOCH 30.00 RESULTS:
 Training Loss:   4.4793 | Training Acc:   9.50% (Hard Labels)
 Validation Loss: 3.8869       | Validation Acc: 13.72% (Actual performance)



# Test

In [6]:
TEST_DATA_PATH = "processed_bird_test_data"
MODEL_FOLDER = "coatnet_aug_experiment" 
MODEL_NAME = "coatnet_0_rw_224"
BATCH_SIZE = 32

MODEL_PATH = f"{MODEL_FOLDER}/best_aug_model" 


if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
test_transforms = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    normalize,
])

def apply_test_transforms(batch):
    batch["pixel_values"] = [test_transforms(image.convert("RGB")) for image in batch["image"]]
    return batch

print(f"Loading test data from '{TEST_DATA_PATH}'.")
try:
    test_dataset = load_from_disk(TEST_DATA_PATH)
    if "test" in test_dataset:
        test_dataset = test_dataset["test"]
        
    print(f"Applying transforms to {len(test_dataset)} images.")
    test_dataset = test_dataset.map(apply_test_transforms, batched=True, batch_size=BATCH_SIZE)
    
    test_dataset.set_format(type="torch", columns=["pixel_values", "id"])
    
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print("Data ready.")
except Exception as e:
    print(f"Error loading data: {e}")
    test_loader = None

if test_loader:
    print(f"Loading best model weights from '{MODEL_PATH}'.")
    model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=200)
    
    try:
        state_dict = safetensors.torch.load_file(f"{MODEL_PATH}/model.safetensors", device=device)
        print("Loaded SafeTensors.")
    except FileNotFoundError:
        state_dict = torch.load(f"{MODEL_PATH}/pytorch_model.bin", map_location=device)
        print("Loaded PyTorch Bin.")
        
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()

    all_preds = []
    all_ids = []

    print("Generating predictions.")
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            pixel_values = batch["pixel_values"].to(device)
            ids = batch["id"]
            
            outputs = model(pixel_values)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            all_ids.extend(ids.numpy())
            
            if i % 20 == 0:
                print(f"Processing batch {i}/{len(test_loader)}.")

    print("\nCreating CSV.")
    submission_df = pd.DataFrame({
        "id": all_ids,
        "label": all_preds
    })

    submission_df["label"] = submission_df["label"] + 1
    
    submission_df = submission_df.sort_values(by="id")

    csv_filename = "coatnet_augmented_submission.csv"
    submission_df.to_csv(csv_filename, index=False)
    
    print(f"Saved '{csv_filename}' successfully!")
    print("\nFirst 5 rows:")
    print(submission_df.head())

Using device: mps
Loading test data from 'processed_bird_test_data'.
Applying transforms to 4000 images.
Data ready.
Loading best model weights from 'coatnet_aug_experiment/best_aug_model'.
Loaded SafeTensors.
Generating predictions.
Processing batch 0/125.
Processing batch 20/125.
Processing batch 40/125.
Processing batch 60/125.
Processing batch 80/125.
Processing batch 100/125.
Processing batch 120/125.

Creating CSV.
Saved 'coatnet_augmented_submission.csv' successfully!

First 5 rows:
   id  label
0   1     17
1   2     35
2   3     40
3   4     12
4   5     32
